In [3]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
%matplotlib inline


import collections
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Image  
from sklearn import tree
import pydotplus
import pandas as pd
import numpy as np


In [4]:
df_2018 = pd.read_csv('ks-projects-201801.csv')


In [5]:
df_2018.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [6]:
df_2018.state.unique()

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

## We are interested in what makes a kickstarter successfull. SO we only care about the successfull or failed states, for if they were cancelled, undefined, suspended we don't know the total project length

In [8]:
df_2018 = pd.read_csv('ks-projects-201801.csv')
df_2018.shape

(378661, 15)

In [9]:
df_2018.state.unique()

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

In [11]:
df_2018['launched_datetime'] = pd.to_datetime(df_2018['launched'])

df_2018['deadline_datetime'] = pd.to_datetime(df_2018['deadline'])

df_2018['project_times'] = pd.to_datetime(df_2018['deadline']) - pd.to_datetime(df_2018['launched'])

'''This extracts the project days from the total project time'''
df_2018['project_length'] = df_2018.project_times.dt.days


In [12]:
df_2018.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,launched_datetime,deadline_datetime,project_times,project_length
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,2015-08-11 12:12:28,2015-10-09,58 days 11:47:32,58
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,2017-09-02 04:43:57,2017-11-01,59 days 19:16:03,59
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,2013-01-12 00:20:50,2013-02-26,44 days 23:39:10,44
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,2012-03-17 03:24:11,2012-04-16,29 days 20:35:49,29
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,2015-07-04 08:35:03,2015-08-29,55 days 15:24:57,55


In [13]:
#Detects the live in state and drops them 
df_2018 = df_2018.loc[df_2018['state']!='live']

#Detects the undefined in state and drops them 
df_2018 = df_2018.loc[df_2018['state']!='undefined']

#Detects the undefined in state and drops them 
df_2018 = df_2018.loc[df_2018['state']!='canceled']

#Detects the undefined in state and drops them 
df_2018 = df_2018.loc[df_2018['state']!='suspended']

#Detects the undefined in usd pledged and drops them 
df_2018 = df_2018.loc[~df_2018['usd pledged'].isna(), :]

#Detects the undefined in usd pledged and drops them 
df_2018 = df_2018.loc[~df_2018['name'].isna(), :]

In [14]:
df_2018.groupby('state').count()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,backers,country,usd pledged,usd_pledged_real,usd_goal_real,launched_datetime,deadline_datetime,project_times,project_length
state,,,,,,,,,,,,,,,,,,
failed,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611,197611
successful,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851,133851


In [20]:
df_2018.project_length = pd.to_numeric(df_2018.project_length)

In [22]:
df_2018.isna().sum()

ID                   0
name                 0
category             0
main_category        0
currency             0
deadline             0
goal                 0
launched             0
pledged              0
state                0
backers              0
country              0
usd pledged          0
usd_pledged_real     0
usd_goal_real        0
launched_datetime    0
deadline_datetime    0
project_times        0
project_length       0
dtype: int64

In [15]:
df_2018.state.value_counts(0)

failed        197611
successful    133851
Name: state, dtype: int64

In [17]:
#number of successful projects
n_projects_successful = df_2018[df_2018['state'] == 'successful'].shape[0]

#number of failed projects
n_projects_failed = df_2018[df_2018['state'] == 'failed'].shape[0]

In [18]:
print(f"Total number of projects: {df_2018.shape[0]}")
print(f"Total number of successful projects: {n_projects_successful}")
print(f"Total number of failed projects: {n_projects_failed}")
print(f'Percentage of successful projects:{(n_projects_successful/projects2018.shape[0])*100}%')

Total number of projects: 331462
Total number of successful projects: 133851
Total number of failed projects: 197611
Percentage of successful projects:35.34850433501205%


In [ ]:
encoder = OneHotEncoder(drop="first", categories="auto").fit([["category", "main_category", "country"]])

In [16]:
projects2018.info()
#catetory, main_category, state, backers, country,usd_goal_real, 
#project_length,name,launched_datetime,deadline_datetime,

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372300 entries, 0 to 378660
Data columns (total 19 columns):
ID                   372300 non-null int64
name                 372296 non-null object
category             372300 non-null object
main_category        372300 non-null object
currency             372300 non-null object
deadline             372300 non-null object
goal                 372300 non-null float64
launched             372300 non-null object
pledged              372300 non-null float64
state                372300 non-null object
backers              372300 non-null int64
country              372300 non-null object
usd pledged          372066 non-null float64
usd_pledged_real     372300 non-null float64
usd_goal_real        372300 non-null float64
launched_datetime    372300 non-null datetime64[ns]
deadline_datetime    372300 non-null datetime64[ns]
project_times        372300 non-null timedelta64[ns]
project_length       372300 non-null int64
dtypes: datetime64[ns](2),

In [17]:
projects2018.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'usd_pledged_real', 'usd_goal_real', 'launched_datetime',
       'deadline_datetime', 'project_times', 'project_length'],
      dtype='object')

In [1]:
projects.state.value_counts

NameError: name 'projects' is not defined

In [18]:
def get_lines():
    with open('ks-projects-201612.csv','rb') as f:
        for line in f:
            yield line

In [ ]:
def get_weird_lines():
    for i, line in enumerate(get_lines()):
        for char in line:
            if char > 127:
                yield line

In [ ]:
weird_lines = get_weird_lines()


In [ ]:
next(weird_lines)

b'1000235643,HIIT Bottle\x99,Drinks,Food,USD,2015-04-27 19:33:08,15000,2015-03-13 18:33:08,124998,successful,2784,US,124998,,,,\r\n'

## encoding is in iso 8859-1

In [ ]:
projects2016 = pd.read_csv('ks-projects-201612.csv',encoding='iso 8859-1')


/Users/krex/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
projects2016.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


## WE  have unnamed columns, let's remove them!

In [ ]:
projects2016.columns

Index(['ID ', 'name ', 'category ', 'main_category ', 'currency ', 'deadline ',
       'goal ', 'launched ', 'pledged ', 'state ', 'backers ', 'country ',
       'usd pledged ', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],
      dtype='object')

## Column names have a space after the last letter, lets get rid of it

In [ ]:
df2016 = projects2016.drop(columns=['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],axis=1)

In [ ]:
df2016 = df2016.rename(columns={'ID ':'ID', 'name ':'name', 'category ':'category', 'main_category ':'main_category',
                       'currency ':'currency', 'deadline ':'deadline','goal ':'goal', 'launched ':'launched',
                       'pledged ':'pledged', 'state ':'state', 'backers ':'backers', 'country ':'country',
                       'usd pledged ':'usd_pledged'})

In [ ]:
df2016.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd_pledged'],
      dtype='object')

In [ ]:
df2016.state.unique()

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended', '4262', '3984', '525', '0', '3035', '3273', '590',
       '23924', '12306', '13675', '2', '4690', '4488.01', '687', '450',
       '16185', '80', '1495', '20', '14309.36', '2011-06-04 01:17:48',
       '8542', '3363', '950', '1800.5', '352', '1320', '18', '2161',
       '2016-06-28 21:36:59', '331', '17861', '1', '2630', '1085', '3711',
       '103', '8609.6', '312', '11044', '5600.65', '630', '1010', '178',
       '2530', '3671', '65', '5', '4451', '16', '2000', '377', '125',
       '6169', '90', '765', '218.5', '1850', '166', '2107', '63', '10',
       '3300', '306', '13', '400', '7703', '650', '800', '4287', '1056',
       '5940', '45', '1827', '10890.45', '245', '276', '200', '30', '6',
       '797', '6126', '175', '12201', '3605.13', '1200', '325', '2632',
       '7271', '46', '2016-02-18 21:40:23', '465', '9430.8', '2308',
       '310', '7470', '2759', '9857.8', '6453', '6189', '6093', '51',
      

In [ ]:
df2016_2 = df2016.loc[(df2016['state'] == 'failed' ) | (df2016['state'] == 'successful')|
                      (df2016['state'] == 'canceled')|(df2016['state'] == 'live')|
                      (df2016['state'] == 'suspended')| (df2016['state'] == 'undefined')]

df2016_nums = pd.concat([df2016,df2016_2]).drop_duplicates(keep=False)


In [ ]:
df2016_2

In [ ]:
df2016_nums

In [ ]:
len(df2016_nums)

In [ ]:
df2016_2.shape #just failed and successful

In [ ]:
df2016_1.shape # failed, successful, and canceled

In [ ]:
df2016.shape[0]-df2016_2.shape[0]

In [ ]:
632/df2016.shape[0] *100

In [ ]:
df2016_2.shape

In [ ]:
type(df2016_2.deadline[0])

In [ ]:
df2016_2.head()

In [ ]:
df2016_2.goal.unique()

## FEATURE SET EXPLORATION
**ID** : continuous  

**name** : title, strings  

**category** : 'Poetry', 'Narrative Film', 'Music', 'Film & Video', 'Restaurants',
       'Food', 'Drinks', 'Product Design', 'Documentary', 'Nonfiction',
       'Indie Rock', 'Crafts', 'Games', 'Design', 'Comic Books',
       'Art Books', 'Fashion', 'Theater', 'Comics', 'DIY', 'Webseries',
       'Food Trucks', 'Public Art', 'Illustration', 'Photography', 'Pop',
       'Tabletop Games', 'Art', 'Family', 'Fiction', 'Accessories',
       'Rock', 'Hardware', 'Weaving', 'Gadgets', 'Web', 'Jazz',
       'Ready-to-wear', 'Festivals', 'Video Games', 'Anthologies',
       'Publishing', 'Shorts', 'Electronic Music', 'Radio & Podcasts',
       'Apps', 'Cookbooks', 'Apparel', 'Metal', 'Hip-Hop', 'Periodicals',
       'Dance', 'Technology', 'Painting', 'Software', 'World Music',
       'Photobooks', 'Drama', 'Architecture', 'Young Adult',
       'Mobile Games', 'Flight', 'Fine Art', 'Action', 'Playing Cards',
       'Makerspaces', 'Punk', "Children's Books", 'People', 'Ceramics',
       'Fabrication Tools', 'Comedy', 'Performances', 'Sculpture',
       'Animation', 'Sound', 'Stationery', "Farmer's Markets", 'Events',
       'Classical Music', 'Graphic Design', 'Spaces', 'Country & Folk',
       'Mixed Media', 'Journalism', 'Movie Theaters', 'Animals',
       'Digital Art', 'Performance Art', 'Installations',
       'Community Gardens', 'DIY Electronics', 'Embroidery',
       'Camera Equipment', 'Farms', 'Conceptual Art', 'Fantasy',
       'Webcomics', 'Experimental', 'Puzzles', 'R&B', 'Print',
       'Music Videos', 'Calendars', 'Vegan', 'Video', 'Plays', 'Blues',
       'Bacon', 'Faith', 'Horror', 'Science Fiction', 'Jewelry',
       'Live Games', 'Small Batch', 'Woodworking', 'Places',
       'Graphic Novels', 'Footwear', '3D Printing', 'Academic',
       'Thrillers', 'Wearables', 'Musical', 'Photo', 'Immersive',
       'Workshops', 'Letterpress', 'Gaming Hardware', 'Zines', 'Candles',
       'Television', 'Audio', 'Space Exploration', 'Couture', 'Nature',
       'Typography', 'Latin', 'Robots', 'Crochet', 'Translations',
       'Textiles', 'Interactive Design', 'Childrenswear', 'Quilts',
       'Pet Fashion', 'Knitting', 'Printing', 'Romance', 'Video Art',
       'Kids', 'Literary Journals', 'Civic Design', 'Glass', 'Pottery',
       'Taxidermy', 'Chiptune', 'Residencies'  
       
**main category** : 'Publishing', 'Film & Video', 'Music', 'Food', 'Design', 'Crafts',
       'Games', 'Comics', 'Fashion', 'Theater', 'Art', 'Photography',
       'Technology', 'Dance', 'Journalism'  
       
**currency** : 'GBP', 'USD', 'CAD', 'NOK', 'AUD', 'EUR', 'MXN', 'SEK', 'NZD',
       'CHF', 'DKK', 'HKD', 'SGD'  
       
**deadline** : date-time
**goal** :   
**launched**  
**pledged**  
**state**  
**backers**  
**country**  
**usd_pledged**  


In [ ]:
launched	pledged	state	backers	country	usd_pledged